In [169]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification

import google_trans_new
from google_trans_new import google_translator  
translator = google_translator()  
import traceback
import logging

In [ ]:
# import sys
# !{sys.executable} -m pip install cellbell

In [73]:
import cellbell

def play_sound(self, etype, value, tb, tb_offset=None):
    %ding
    self.showtraceback((etype, value, tb), tb_offset=tb_offset)
    print('ding worked!')

C:\Users\aligo\Documents\DEV\twitter-lv-market\venv38\lib\site-packages\pyglet\media\codecs\wmf.py:838: UserWarning: [WinError -2147417850] Cannot change thread mode after it is set
  warnings.warn(str(err))


In [74]:
get_ipython().set_custom_exc((Exception,), play_sound)

In [78]:
1/0

ZeroDivisionError: division by zero

ding worked!


In [77]:
# df = pd.read_csv('./../Naive_Bayes/tweets/training1600000processednoemoticon.csv', names=['target','id','date','flag','user','text'])
# df.head()

In [51]:
# df['target'].value_counts()

In [31]:
df = pd.read_csv('./../Naive_Bayes/tweets/p_n_n_dataset.csv')
df.shape[0]

174644

In [3]:
df['label'].value_counts()

neutral     77028
positive    51994
negative    45622
Name: label, dtype: int64

In [54]:
label_dict = {'neutral': 0, 'positive': 1, 'negative': 2}
df['label'] = df.label.replace(label_dict) 
df['label'].value_counts()

0    77028
1    51994
2    45622
Name: label, dtype: int64

In [55]:
# translate_text = translator.translate(df.loc[1]['tweet']+'    ', lang_src='en', lang_tgt='lv')


# print(df.loc[1]['tweet'])
# print(translate_text)

In [3]:
df['message_lv_google']=pd.Series(dtype='float64')
df[pd.notnull(df['message_lv_google'])].size

0

In [174]:
df = pd.read_csv('./../Naive_Bayes/tweets/df_google_lv_1.csv')
df.shape[0]

174644

In [79]:
# start = 54744
# google_new_transError:

In [161]:
start

70651

In [168]:
df.iloc[start-1]

id                                               1290000000000000000.0
date                                                         7/22/2020
time                                                           5:43:32
user_id                                                    135276057.0
username                                                    ivelisse_a
name                                                 Poisonivy_A🌊🇵🇷🇺🇸🌊
tweet                @mattgaetz @realDonaldTrump Wtf, Liz Cheney re...
language                                                            en
label                                                                1
message_lv_google    Kur bija šis Patriks Beverly? Viņš spēlē ar ti...
Name: 71874, dtype: object

In [150]:
df.iloc[start]

id                                               1080000000000000000.0
date                                                         1/10/2019
time                                                          13:06:48
user_id                                           854000000000000000.0
username                                                  sabialatina7
name                                                       Maria Monge
tweet                Women's Ministries Fervent Friday's January 18...
language                                                            en
label                                                                1
message_lv_google    Kur bija šis Patriks Beverly? Viņš spēlē ar ti...
Name: 68108, dtype: object

In [151]:
start+3000

71108

In [152]:
df_1 = df.iloc[start:(start+3000)]
df_1.shape[0]

3000

In [153]:
eng_to_lv = []
for index, row in df_1.iterrows():
    try:
        eng_to_lv.append(translator.translate(row['tweet'].strip(), lang_src='en', lang_tgt='lv'))
        if (index%100) == 0:
            print(index)
    except TypeError:
        print(row['tweet'])
        eng_to_lv.append(row['tweet'].strip())

68200
68300
68400
68500
68600
68700
68800
68900
69000
69100
69200
69300
69400
69500
69600
69700
69800
69900
70000
70100
70200
70300
70400
70500
70600


NameError: name 'google_new_transError' is not defined

ding worked!


In [154]:
len(eng_to_lv)

2543

In [155]:
start

68108

In [166]:
index = start

for translation in eng_to_lv:
    df['message_lv_google'][index] = translation
    index = index+1

C:\Users\aligo\AppData\Local\Temp/ipykernel_7136/151635087.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['message_lv_google'][index] = translation


In [157]:
df.to_csv('./../Naive_Bayes/tweets/df_google_lv.csv', index=False)

In [158]:
start = start + len(eng_to_lv)

In [159]:
insurance = []

In [170]:
def translateGoogle(start):
    print('Start:')
    print(start)
    df_1 = df.iloc[start:(start+3000)]
    eng_to_lv = []
    for index, row in df_1.iterrows():
        try:
            if (index<174645):
                eng_to_lv.append(translator.translate(row['tweet'].strip(), lang_src='en', lang_tgt='lv'))
                insurance.append(eng_to_lv[len(eng_to_lv)-1])
                if (index%100) == 0:
                    print(index)
        except TypeError:
            print(row['tweet'])
            eng_to_lv.append(row['tweet'].strip())
        except Exception as e:
            logging.error(traceback.format_exc())
            print('Timeout Exception!')
            index = start
            # save to file
            for translation in eng_to_lv:
                df['message_lv_google'][index] = translation
                index = index+1
            df.to_csv('./../Naive_Bayes/tweets/df_google_lv.csv', index=False)
            # repeat
            start = start + len(eng_to_lv)
            if(start<174644):
                translateGoogle(start)

In [171]:
translateGoogle(71874)

Start:
71874
71900
72000
72100
72200
@BruceShelkey No matter what you charge no matter how quickly you do it some of them will never be grateful.... I'm so glad I'm an electrical guy I could never deal with plumbing...
72300
72400
72500
72600


ERROR:root:Traceback (most recent call last):
  File "C:\Users\aligo\Documents\DEV\twitter-lv-market\venv38\lib\site-packages\urllib3\connectionpool.py", line 382, in _make_request
    self._validate_conn(conn)
  File "C:\Users\aligo\Documents\DEV\twitter-lv-market\venv38\lib\site-packages\urllib3\connectionpool.py", line 1010, in _validate_conn
    conn.connect()
  File "C:\Users\aligo\Documents\DEV\twitter-lv-market\venv38\lib\site-packages\urllib3\connection.py", line 416, in connect
    self.sock = ssl_wrap_socket(
  File "C:\Users\aligo\Documents\DEV\twitter-lv-market\venv38\lib\site-packages\urllib3\util\ssl_.py", line 449, in ssl_wrap_socket
    ssl_sock = _ssl_wrap_socket_impl(
  File "C:\Users\aligo\Documents\DEV\twitter-lv-market\venv38\lib\site-packages\urllib3\util\ssl_.py", line 493, in _ssl_wrap_socket_impl
    return ssl_context.wrap_socket(sock, server_hostname=server_hostname)
  File "C:\Users\aligo\AppData\Local\Programs\Python\Python38\lib\ssl.py", line 500, in wrap_

Timeout Exception!
Start:
72649
72700
72800
72900
73000
73100
73200
73300
73400
73500
73600
73700
73800
73900
74000
74100
74200
74300
74400
74500
74600
74700
74800
74900


ERROR:root:Traceback (most recent call last):
  File "C:\Users\aligo\Documents\DEV\twitter-lv-market\venv38\lib\site-packages\urllib3\connectionpool.py", line 382, in _make_request
    self._validate_conn(conn)
  File "C:\Users\aligo\Documents\DEV\twitter-lv-market\venv38\lib\site-packages\urllib3\connectionpool.py", line 1010, in _validate_conn
    conn.connect()
  File "C:\Users\aligo\Documents\DEV\twitter-lv-market\venv38\lib\site-packages\urllib3\connection.py", line 416, in connect
    self.sock = ssl_wrap_socket(
  File "C:\Users\aligo\Documents\DEV\twitter-lv-market\venv38\lib\site-packages\urllib3\util\ssl_.py", line 449, in ssl_wrap_socket
    ssl_sock = _ssl_wrap_socket_impl(
  File "C:\Users\aligo\Documents\DEV\twitter-lv-market\venv38\lib\site-packages\urllib3\util\ssl_.py", line 493, in _ssl_wrap_socket_impl
    return ssl_context.wrap_socket(sock, server_hostname=server_hostname)
  File "C:\Users\aligo\AppData\Local\Programs\Python\Python38\lib\ssl.py", line 500, in wrap_

Timeout Exception!
Start:
74926
75000
75100
75200
75300
75400
75500
75600
75700
75800
75900
76000
76100
76200
76300
76400
76500
76600
76700
76800
76900
77000
77100
77200


...

ERROR:root:Traceback (most recent call last):
  File "C:\Users\aligo\Documents\DEV\twitter-lv-market\venv38\lib\site-packages\urllib3\connectionpool.py", line 445, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "C:\Users\aligo\Documents\DEV\twitter-lv-market\venv38\lib\site-packages\urllib3\connectionpool.py", line 440, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Users\aligo\AppData\Local\Programs\Python\Python38\lib\http\client.py", line 1344, in getresponse
    response.begin()
  File "C:\Users\aligo\AppData\Local\Programs\Python\Python38\lib\http\client.py", line 307, in begin
    version, status, reason = self._read_status()
  File "C:\Users\aligo\AppData\Local\Programs\Python\Python38\lib\http\client.py", line 268, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "C:\Users\aligo\AppData\Local\Programs\Python\Python38\lib\socket.py", line 669, in readinto
    return self._sock

Timeout Exception!
Start:
121002


ERROR:root:Traceback (most recent call last):
  File "C:\Users\aligo\Documents\DEV\twitter-lv-market\venv38\lib\site-packages\urllib3\connectionpool.py", line 445, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "C:\Users\aligo\Documents\DEV\twitter-lv-market\venv38\lib\site-packages\urllib3\connectionpool.py", line 440, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Users\aligo\AppData\Local\Programs\Python\Python38\lib\http\client.py", line 1344, in getresponse
    response.begin()
  File "C:\Users\aligo\AppData\Local\Programs\Python\Python38\lib\http\client.py", line 307, in begin
    version, status, reason = self._read_status()
  File "C:\Users\aligo\AppData\Local\Programs\Python\Python38\lib\http\client.py", line 268, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "C:\Users\aligo\AppData\Local\Programs\Python\Python38\lib\socket.py", line 669, in readinto
    return self._sock

Timeout Exception!


KeyboardInterrupt: 

In [165]:
len(insurance)

1223

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

df.groupby(['label', 'data_type']).count()

id   date   time  user_id  username   name  tweet  \
label data_type                                                         
0     train      65474  65474  65474    65474     65474  65473  65474   
      val        11554  11554  11554    11554     11554  11554  11554   
1     train      44195  44195  44195    44195     44195  44195  44195   
      val         7799   7799   7799     7799      7799   7799   7799   
2     train      38778  38778  38778    38778     38778  38777  38778   
      val         6844   6844   6844     6844      6844   6843   6844   

                 language  message_eng  
label data_type                         
0     train         65474        65474  
      val           11554        11554  
1     train         44195        44195  
      val            7799         7799  
2     train         38778        38778  
      val            6844         6844

In [7]:
df[df.data_type=='train']['label'].value_counts()

0    65474
1    44195
2    38778
Name: label, dtype: int64

In [8]:
df_val = [df[df.data_type=='val'].message_eng, df[df.data_type=='val'].label]

df_train = [df[df.data_type=='train'].message_eng, df[df.data_type=='train'].label]
df_train = pd.concat(df_train, axis=1, keys=["message_eng", "label"])

df_0 = df_train[df_train['label']==0]
df_1 = df_train[df_train['label']==1]
df_2 = df_train[df_train['label']==2]

df_0_downsampled = df_0.sample(df_2.shape[0])
df_1_downsampled = df_1.sample(df_2.shape[0])

df_train = pd.concat([df_0_downsampled, df_1_downsampled, df_2])

df_train['label'].value_counts()

0    38778
1    38778
2    38778
Name: label, dtype: int64

In [9]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', 
                                          do_lower_case=True)
                                          
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].message_eng.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].message_eng.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\aligo\Documents\DEV\twitter-lv-market\venv38\lib\site-packages\transformers\tokenization_utils_base.py:2212: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [10]:
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased",
                                                      num_labels=3,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cpu


In [13]:
model.load_state_dict(torch.load('models4/finetuned_BERT_ENG_epoch_1.model', map_location=device))

<All keys matched successfully>

In [15]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [16]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [17]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict = {0: 0, 1: 1, 2: 2,}
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [18]:
import random
import numpy as np

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [19]:
epoch = 2

In [20]:
while epoch < (epochs+1):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'models4/finetuned_BERT_ENG_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')
    
    epoch = epoch + 1

Epoch 2:   0%|          | 0/49483 [00:00<?, ?it/s]

KeyboardInterrupt: 